# Derive terrain attributes from a Digital Elevation Model (DEM)

## Description
The purpose of this notebook is to compute terrain attributes from a Digital Elevation Model (DEM) for a specific area of interest. These terrain attributes provide important information about the topographic characteristics of the landscape and are particularly useful for studying wetlands. The notebook calculates various terrain indices such as aspect, curvature, Topographic Wetness Index (TWI), Terrain Profile Index (TPI), and hillshade. These indices offer insights into slope, orientation, shape, hydrology, water flow patterns, and other factors relevant to wetlands. By deriving these terrain attributes, researchers and analysts can better understand wetland dynamics, assess habitats, model ecosystems, and plan conservation strategies. The resulting terrain indices enhance wetland mapping, classification, and analysis, facilitating more accurate and detailed studies related to wetland ecosystems.

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Inastall and load packages" cell.

## Install and load Packages

### Optional - Install onetime

In [1]:
# !pip install richdem
# !pip install xarray-spatial
# !pip install focal_stats

In [2]:
%matplotlib inline
import os
import math
import datacube
import warnings
import rioxarray
import richdem as rd
import numpy as np
import geopandas as gpd
import rasterio as rio
from xrspatial import focal
import matplotlib.pyplot as plt
from xrspatial import hillshade
from xrspatial import convolution
from datacube.utils import geometry
from odc.dscache.tools import tiling
from datacube.utils.geometry import BoundingBox, Geometry
from datashader.transfer_functions import shade
from datacube.utils.geometry import Geometry
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.areaofinterest import define_area

import warnings

warnings.filterwarnings("ignore")

### Set up a dask cluster
This will help keep our memory use down and conduct the analysis in parallel. If you'd like to view the dask dashboard, click on the hyperlink that prints below the cell. You can use the dashboard to monitor the progress of calculations.

In [3]:
create_local_dask_cluster()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/mpho.sadiki@digitalearthafrica.org/proxy/8787/status,
Dashboard: /user/mpho.sadiki@digitalearthafrica.org/proxy/8787/status,Workers: 1
Total threads: 4,Total memory: 26.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46247,Workers: 1
Dashboard: /user/mpho.sadiki@digitalearthafrica.org/proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 26.21 GiB
Comm: tcp://127.0.0.1:34363,Total threads: 4
Dashboard: /user/mpho.sadiki@digitalearthafrica.org/proxy/42333/status,Memory: 26.21 GiB
Nanny: tcp://127.0.0.1:45327,


### Initialize Datacube

In [4]:
dc = datacube.Datacube(app="DEM")

### Load vector and plot area of interest

In [5]:
# Specify a prefix to identify the area of interest in the saved outputs
# By assigning the desired prefix, you can easily identify the outputs associated with the specific area of interest.
prefix = "South_Africa"

aoi = define_area(shapefile_path='data/south_africa.geojson')

# Create a geopolygon and geodataframe of the area of interest
geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")

### Explore the product names, resolution, and measurements

In [7]:
product_name = ['dem_cop_30', 'dem_srtm_deriv']
resolution = (-90, 90)
measurements = 'elevation'
dc_measurements = dc.list_measurements()
dc_measurements.loc[product_name].drop('flags_definition', axis=1)

name    dtype units   nodata aliases
product        measurement                                           
dem_cop_30     elevation    elevation  float32     1      NaN     NaN
dem_srtm_deriv mrvbf            mrvbf    int16     1 -32768.0     NaN
               mrrtf            mrrtf    int16     1 -32768.0     NaN
               slope            slope  float32     1  -9999.0     NaN

### Set up reusable  query object 

In [8]:
dask_chunks = {'x': 2500, 'y': 2500}

# set up daatcube query object
query = {
    'resolution': resolution,
    'output_crs': 'epsg:6933',
    "geopolygon": geopolygon,
    'dask_chunks': dask_chunks
}

### Derive terrain attributes

##### Elevation, slope, MrVBF, MrRTF, Aspect and curvature

In [9]:
# Define the output directory
output_dir = "data/terrain_attributes/"
# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Load the dem 30 m product
ds_elev = dc.load(product="dem_cop_30", measurements='elevation',
                  **query).squeeze()
# Load the ds_deriv dataset
ds_deriv = dc.load(product="dem_srtm_deriv",
                   measurements=['mrvbf', 'mrrtf', 'slope'],
                   **query).squeeze()

In [10]:
elevation_path = os.path.join(output_dir, f"{prefix}_elevation.tif")
ds_elev.elevation.rio.to_raster(elevation_path,
                                compress="deflate",
                                compress_opts=dict(zlevel=6))

# Open the input elevation GeoTIFF file to get the profile information
with rio.open(elevation_path) as src:
    profile = src.profile

    # Update the profile with the desired compression
    profile.update(compress="deflate", compress_opts=dict(zlevel=6))

    # Calculate terrain attributes
    elevation = rd.rdarray(ds_elev.elevation, no_data=-9999)
    aspect = rd.TerrainAttribute(elevation, attrib='aspect')
    curvature = rd.TerrainAttribute(elevation, attrib='curvature')
    profile_curvature = rd.TerrainAttribute(elevation,
                                            attrib='profile_curvature')
    planform_curvature = rd.TerrainAttribute(elevation,
                                             attrib='planform_curvature')

    # Export other terrain attributes using the elevation's profile information
    attributes = [(aspect, f"{prefix}_aspect.tif"),
                  (curvature, f"{prefix}_curvature.tif"),
                  (profile_curvature, f"{prefix}_profile_curvature.tif"),
                  (planform_curvature, f"{prefix}_planform_curvature.tif")]

    for attribute, output_filename in attributes:
        output_path = os.path.join(output_dir, output_filename)

        # Create the output GeoTIFF file and write the attribute data
        with rio.open(output_path, 'w', **profile, quiet=True) as dst:
            dst.write(attribute, 1)

# Export slope, mrrtf, and mrvbf using the elevation's profile information
slope_path = os.path.join(output_dir, f"{prefix}_slope.tif")
ds_deriv.slope.rio.to_raster(slope_path, **profile)

mrrtf_path = os.path.join(output_dir, f"{prefix}_mrrtf.tif")
ds_deriv.mrrtf.rio.to_raster(mrrtf_path, **profile)

mrvbf_path = os.path.join(output_dir, f"{prefix}_mrvbf.tif")
ds_deriv.mrvbf.rio.to_raster(mrvbf_path, **profile)

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)



A Aspect attribute calculation
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

[==                                                ] (3% - 490.1s - 1 threads))

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)


t Wall-time = 23.1439                         ] (3% - 745.3s - 1 threads))

A Curvature attribute calculation
C Zevenbergen, L.W., Thorne, C.R., 1987. Quantitative analysis of land surface topography. Earth surface processes and landforms 12, 47–56.

t Wall-time = 7.68431                         ] (3% - 247.2s - 1 threads)

A Profile curvature attribute calculation
C Zevenbergen, L.W., Thorne, C.R., 1987. Quantitative analysis of land surface topography. Earth surface processes and landforms 12, 47–56.

t Wall-time = 15.8445                         ] (3% - 509.8s - 1 threads))

A Planform curvature attribute calculation
C Zevenbergen, L.W., Thorne, C.R., 1987. Quantitative analysis of land surface topography. Earth surface processes and landforms 12, 47–56.

t Wall-time = 15.8484                         ] (3% - 509.9s - 1 threads))


#### Topographic Wetness Index (TWI) and Topographic Postion Index (TPI)

In [ ]:
ma_elevation = rd.rdarray(ds_elev.elevation,no_data=-9999)
# Derive Topographic Wetness Index (TWI)

terrain = ds_elev.elevation
# 2Pi radians = 360 degrees
# Pi radians = 180 degrees
# conversion: Pi radians/180 degress
slope_radians = ds_deriv.slope * math.pi/180.0

# Get flow accumulation with no explicit weighting. The default will be 1.
accum_d8 = rd.FlowAccumulation(ma_elevation, method='D8')

# TWI - Every cell calculate the upslope contributing area - stream
twi = np.log(accum_d8 / (np.tan(slope_radians)+.01))


# Derive Topographic Position Index (TPI)   and hillsahde
cellsize_x, cellsize_y = convolution.calc_cellsize(terrain)

# Use an annulus kernel with a ring at a distance from 25-30 cells away from focal point
outer_radius = str(cellsize_x * 30) + "m"
inner_radius = str(cellsize_x * 25) + "m"
kernel = convolution.annulus_kernel(
    cellsize_x, cellsize_y, outer_radius, inner_radius)

tpi = terrain - focal.apply(terrain, kernel)

tpi_terrain = hillshade(terrain - focal.apply(terrain, kernel))
tpi_terrain_shaded = shade(
    tpi_terrain, cmap=["white", "black"], alpha=255, how="linear"
)


# Export TWI, TPI and Hillshade
# Open the input elevation GeoTIFF file to get the profile information
with rio.open(os.path.join(output_dir, f"{prefix}_elevation.tif")) as src:
    profile = src.profile

    # Update the profile with the desired compression
    profile.update(compress="deflate", compress_opts=dict(level=6))

    # Export each variable as a separate GeoTIFF file
    for variable, variable_name in [
        (tpi, f"{prefix}_TPI.tif"),
        (twi,f"{prefix}_TWI.tif"),
        (tpi_terrain, f"{prefix}_hillshade.tif"),
    ]:
        # Generate the output file path
        output_file = os.path.join(output_dir, variable_name)

        # Create the output GeoTIFF file and write the variable data
        with rio.open(output_file, 'w', **profile) as dst:
            dst.write(variable, 1)

/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')